Homework for https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md

In [1]:
import os
import requests
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

/Users/vladkha/Desktop/Workspace/Coding/Online_courses/DataTalksClub/LLM_Zoomcamp/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Q1

In [2]:
!curl localhost:9200 | grep build_hash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   539  100   539    0     0   6785      0 --:--:-- --:--:-- --:--:--  6822
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",


In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2

In [4]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [5]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 327.51it/s]


## Q3

In [6]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }
es_client.search(index=index_name, body=search_query)

ObjectApiResponse({'took': 24, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions', '_id': 'lS8pXJAB1J9DpHr8buaV', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}]}})

## Q4

In [7]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": 'machine-learning-zoomcamp'
                    }
                }
            }
        }
    }
search_results = [h['_source'] for h in es_client.search(index=index_name, body=search_query)['hits']['hits']]
for r in search_results:
    print(r['question'])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


## Q5

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    context_template = """
Q: {question}
A: {text}
""".strip()
    for doc in search_results:
        context += context_template.format(question=doc['question'], text=doc['text']) + '\n\n'
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [9]:
prompt = build_prompt(query=query, search_results=search_results)
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [10]:
len(prompt)

1462

## Q6

In [13]:
# !pip install tiktoken

In [14]:
import tiktoken

In [15]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [16]:
len(encoding.encode(prompt))

322

## Bonus: generating the answer (ungraded)

In [17]:
# add your HF token as an environment variable via export HF_TOKEN='hf_YOUR_TOKEN' 
# (see https://huggingface.co/docs/hub/en/security-tokens)
HF_API_HEADERS = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}
HF_API_URL = "https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct"

def llm(prompt):
    response = requests.post(HF_API_URL, headers=HF_API_HEADERS, json={"inputs": prompt})
    result = response.json()
    result = result[0]['generated_text']
    result = result[result.index(prompt) + len(prompt):]
    return result.strip()

In [18]:
output = llm(prompt)
print(output)

A: How do I execute a command in a running docker container?

ANSWER: If the container is already running, execute a command in the specific container: docker exec -it <container-id> bash. Replace <container-id> with the id of the running container found using "docker ps".


## Bonus: calculating the costs (ungraded)

In [19]:
average_input_len = 150
average_output_len = 250
input_token_price = 0.005 / 1000
output_token_price = 0.015 / 1000
num_requests = 1000

average_request_price = average_input_len * input_token_price + average_output_len * output_token_price
average_request_price * num_requests

4.5

In [20]:
q6_input_len = len(encoding.encode(prompt))
q7_output_len = len(encoding.encode(output))

q_price = q6_input_len * input_token_price + q7_output_len * output_token_price
q_price

0.002495